In [30]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
from nba_api.stats.static import teams
import pandas as pd
import time
import os

In [2]:
nba_players = players.get_players()
nba_teams = teams.get_teams()

In [24]:
# Find the player ID for LeBron James.
lbj = [player for player in nba_players if player['full_name'] == 'LeBron James'][0]

In [25]:
playercareerstats.PlayerCareerStats(player_id=lbj['id']).get_data_frames()[0].head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,2544,2003-04,00,1610612739,CLE,19.0,79,79,3120.0,622,...,0.754,99,333,432,465,130,58,273,149,1654
1,2544,2004-05,00,1610612739,CLE,20.0,80,80,3388.0,795,...,0.750,111,477,588,577,177,52,262,146,2175
2,2544,2005-06,00,1610612739,CLE,21.0,79,79,3361.0,875,...,0.738,75,481,556,521,123,66,260,181,2478
3,2544,2006-07,00,1610612739,CLE,22.0,78,78,3190.0,772,...,0.698,83,443,526,470,125,55,250,171,2132
4,2544,2007-08,00,1610612739,CLE,23.0,75,74,3027.0,794,...,0.712,133,459,592,539,138,81,255,165,2250


In [26]:
active_nba_players = [player for player in nba_players if player['is_active'] == True]

In [39]:
# 取得 2022-23 賽季，所有球員的資料
player_2022_23_stats = pd.DataFrame()

t0 = time.time()
for player in active_nba_players:
    t1 = time.time()
    player_id = player['id']
    player_stats = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]
    t2 = time.time()
    player_2022_23_stats = pd.concat([player_2022_23_stats, player_stats[player_stats['SEASON_ID'] == '2022-23']], ignore_index=False)
    t3 = time.time()

    with open('player_stats.csv', 'w') as file:
        file.write('player_id, time\n')
        file.write('API Time: '+ str(t2 - t1) +'\n')
        file.write('AGG Time: '+ str(t3 - t2) +'\n')
        file.write('Loop Time: '+ str(t3 - t1) +'\n')


    time.sleep(0.5)

In [45]:
player_2022_23_stats['PLAYER_ID'].unique().shape

(527,)

In [50]:
player_id_list = []
for player in active_nba_players:
    player_id = player['id']
    player_id_list.append(player_id)

In [53]:
player_disappear = set(player_id_list) - set(player_2022_23_stats['PLAYER_ID'].unique())

In [54]:
# 確認為登錄球員卻沒有資料的球員
for player in active_nba_players:
    if player['id'] in player_disappear:
        print(player['full_name'])

Ibou Badji
Lonzo Ball
Danilo Gallinari
Collin Gillespie
Chet Holmgren
DaQuan Jeffries
Justin Lewis
E.J. Liddell


In [55]:
player_2022_23_stats.to_csv('player_2022_23_stats.csv', index=False)

# NBA_API Data 確認

## All time leader

In [67]:
from nba_api.stats.endpoints import alltimeleadersgrids

In [69]:
alltimeleaders = alltimeleadersgrids.AllTimeLeadersGrids()

In [78]:
alltimeleaders.get_request_url()

'https://stats.nba.com/stats/alltimeleadersgrids?LeagueID=00&PerMode=Totals&SeasonType=Regular+Season&TopX=10'

In [74]:
len(alltimeleaders.get_data_frames())

19

In [83]:
for data in alltimeleaders.get_data_frames():
    print(data.head())
    print('--------------')

   PLAYER_ID          PLAYER_NAME    GP  GP_RANK IS_ACTIVE_FLAG
0        305        Robert Parish  1611        1              N
1      76003  Kareem Abdul-Jabbar  1560        2              N
2       1713         Vince Carter  1541        3              N
3       1717        Dirk Nowitzki  1522        4              N
4        304        John Stockton  1504        5              N
--------------
   PLAYER_ID          PLAYER_NAME    PTS  PTS_RANK IS_ACTIVE_FLAG
0       2544         LeBron James  38652         1              Y
1      76003  Kareem Abdul-Jabbar  38387         2              N
2        252          Karl Malone  36928         3              N
3        977          Kobe Bryant  33643         4              N
4        893       Michael Jordan  32292         5              N
--------------
   PLAYER_ID    PLAYER_NAME    AST  AST_RANK IS_ACTIVE_FLAG
0        304  John Stockton  15806         1              N
1        467     Jason Kidd  12091         2              N
2     1011

## Assist leader

In [85]:
from nba_api.stats.endpoints import assistleaders

In [86]:
assist_leader = assistleaders.AssistLeaders()

In [91]:
for data in assist_leader.get_data_frames():
    print(data.head())
    print('--------------')

   RANK     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME   AST
0     1  1610612744               GSW  Golden State Warriors  2446
1     2  1610612743               DEN         Denver Nuggets  2368
2     3  1610612758               SAC       Sacramento Kings  2237
3     4  1610612756               PHX           Phoenix Suns  2235
4     5  1610612759               SAS      San Antonio Spurs  2232
--------------


## Assist tracker

In [94]:
from nba_api.stats.endpoints import assisttracker

In [96]:
assist_tracker = assisttracker.AssistTracker()
for data in assist_tracker.get_data_frames():
    print(data.head())
    print('--------------')

   ASSISTS
0    57727
--------------


In [105]:
assist_tracker.get_dict()

{'resource': 'assisttracker',
 'parameters': {'PerMode': None,
  'LeagueID': None,
  'Season': None,
  'SeasonType': None,
  'PORound': None,
  'Outcome': None,
  'Location': None,
  'Month': None,
  'SeasonSegment': None,
  'DateFrom': None,
  'DateTo': None,
  'OpponentTeamID': None,
  'VsConference': None,
  'VsDivision': None,
  'TeamID': None,
  'Conference': None,
  'Division': None,
  'LastNGames': None,
  'GameScope': None,
  'PlayerExperience': None,
  'PlayerPosition': None,
  'StarterBench': None,
  'DraftYear': None,
  'DraftPick': None,
  'College': None,
  'Country': None,
  'Height': None,
  'Weight': None},
 'resultSets': [{'name': 'AssistTracker',
   'headers': ['ASSISTS'],
   'rowSet': [[57727]]}]}

## BoxScoreAdvancedV2

In [106]:
from nba_api.stats.endpoints import boxscoreadvancedv2

In [111]:
boxscroe_advanced = boxscoreadvancedv2.BoxScoreAdvancedV2(end_period='0', game_id='0022000001', start_period='0')

In [112]:
boxscore_advanced_df = boxscroe_advanced.get_data_frames()

In [114]:
boxscore_advanced_df[0]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022000001,1610612744,GSW,Golden State,203952,Andrew Wiggins,Andrew,F,,31.000000:14,...,17.4,0.313,0.366,0.259,0.262,111.88,109.11,90.93,72,-0.044
1,0022000001,1610612744,GSW,Golden State,1629672,Eric Paschall,Eric,F,,21.000000:33,...,0.0,0.417,0.436,0.127,0.130,112.22,111.37,92.81,49,0.012
2,0022000001,1610612744,GSW,Golden State,1630164,James Wiseman,James,C,,24.000000:17,...,5.6,0.577,0.575,0.286,0.290,116.62,114.65,95.54,58,0.166
3,0022000001,1610612744,GSW,Golden State,1626162,Kelly Oubre Jr.,Kelly,G,,25.000000:39,...,15.8,0.214,0.214,0.243,0.244,120.78,118.83,99.03,63,0.000
4,0022000001,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,30.000000:19,...,8.3,0.381,0.439,0.313,0.314,119.41,117.16,97.64,74,0.127
5,0022000001,1610612744,GSW,Golden State,1626172,Kevon Looney,Kevon,,,11.000000:17,...,33.3,0.500,0.450,0.194,0.192,114.01,116.99,97.49,27,-0.036
6,0022000001,1610612744,GSW,Golden State,203145,Kent Bazemore,Kent,,,11.000000:35,...,33.3,0.500,0.500,0.121,0.120,119.34,120.17,100.14,29,0.020
7,0022000001,1610612744,GSW,Golden State,1627814,Damion Lee,Damion,,,12.000000:14,...,0.0,0.750,0.750,0.057,0.058,110.41,111.83,93.19,29,0.216
8,0022000001,1610612744,GSW,Golden State,1627737,Marquese Chriss,Marquese,,,12.000000:26,...,0.0,0.450,0.450,0.294,0.296,107.32,104.24,86.86,27,0.205
9,0022000001,1610612744,GSW,Golden State,1629673,Jordan Poole,Jordan,,,17.000000:55,...,22.2,0.250,0.250,0.133,0.135,103.63,103.14,85.95,38,0.000


In [115]:
boxscore_advanced_df[1]

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022000001,1610612744,Warriors,GSW,Golden State,240.000000:00,86.8,88.4,110.5,111.6,...,16.1,0.424,0.454,1.0,0.201,113.6,112.0,93.33,112,0.35
1,0022000001,1610612751,Nets,BKN,Brooklyn,240.000000:00,110.5,111.6,86.8,88.4,...,17.9,0.538,0.589,1.0,0.201,113.6,112.0,93.33,112,0.65


## boxscoredefensive